In [ ]:
import polyscope as ps
import matplotlib.colors as mcolors
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
def draw_grid(grid_colors):
    """
    Draws the face of a hue cube in a grid.

    grid_colors -- a np array of shape (n, n, n_channels); n_channels may be either 3 or 4
    """
    grid_colors = np.flip(grid_colors, 1)
    n, _, n_channels = grid_colors.shape
    
    final_colors = np.empty((3 * n, 3 * n, n_channels))

    for i in [0, 2]:
        for j in [0, 2]:
            row, col = n * i, n * j
            final_colors[row:row+n, col:col+n] = grid_colors

    for i in [1]:
        for j in [0, 2]:
            row, col = n * i, n * j
            final_colors[row:row+n, col:col+n] = np.flip(grid_colors, 0)

    for i in [0, 2]:
        for j in [1]:
            row, col = n * i, n * j
            final_colors[row:row+n, col:col+n] = np.flip(grid_colors, 1)

    for i in [1]:
        for j in [1]:
            row, col = n * i, n * j
            final_colors[row:row+n, col:col+n] = np.flip(grid_colors, [0, 1])

    final_colors = final_colors.reshape(-1, n_channels)

    # Point cloud geometry
    xs = np.arange(0, 3 * n)
    ys = np.arange(0, 3 * n)

    xx, yy = np.meshgrid(xs, ys)
    pos = np.stack([xx, yy], axis=-1).reshape(-1, 2)
    pos = np.flip(pos, [0, 1])

    cloud = ps.register_point_cloud('grid', points=pos)
    cloud.set_radius(0.45, relative=False)
    if n_channels == 3:
        cloud.add_color_quantity('colors', final_colors, enabled=True)
        cloud.set_material('flat')
    elif n_channels == 4:
        cloud.add_tetracolor_quantity('colors', final_colors, enabled=True)

In [ ]:
def animate_grid(z, tetra_image_mode=None):
    if tetra_image_mode is None:
        fd = ps.open_video_file('./out/tri/rgb_square.mp4')
    else:
        fd = ps.open_tetra_video_file('./out/LMS_Q/rgb_square.mp4', save_image_mode=tetra_image_mode)

    n = 5
    key_positions = [
        [2 * n + n // 2, n + n // 2, z],
        # [2 * n + n // 2, 2 * n + n // 2, z],
        # [n + n // 2, 2 * n + n // 2, z],
        # [n // 2, 2 * n + n // 2, z],
        # [n // 2, n + n // 2, z],
        # [n // 2, n // 2, z],
        # [n + n // 2, n // 2, z],
        # [n + n // 2, n + n // 2, z]
    ]
    frames_per_pos = 240

    for pos in key_positions:
        curr_cam_pos = ps.get_view_camera_parameters().get_position()
        for i in range(frames_per_pos):
            next_cam_pos = curr_cam_pos + (i / frames_per_pos) * (pos - curr_cam_pos)
            target = np.copy(next_cam_pos)
            target[-1] = 0
            ps.look_at(next_cam_pos, target)

            if tetra_image_mode is None:
                ps.write_video_frame(fd)
            else:
                ps.write_tetra_video_frame(fd)
    
    if tetra_image_mode is None:
        ps.close_video_file(fd)
    else:
        ps.close_tetra_video_file(fd)

In [ ]:
# Bounding box depends on grid size
ps.set_automatically_compute_scene_extents(False)
ps.set_length_scale(1.)

# Camera settings
fov = 30.0
intrinsics = ps.CameraIntrinsics(fov_vertical_deg=fov,
                                    fov_horizontal_deg=fov,
                                    aspect=1.)

n = 5
center = n + n // 2
z = 0.5 * n / np.tan(np.radians(0.5 * fov))
cam_pos = (center, center, z)
extrinsics = ps.CameraExtrinsics(root=cam_pos,
                                    look_dir=(0, 0, -1),
                                    up_dir=(0, 1, 0))
params = ps.CameraParameters(intrinsics, extrinsics)
ps.set_view_camera_parameters(params)

n_channels = 4
if n_channels == 3:
    n_colors = n ** 2
    hues = np.linspace(0.5, 0.85, n_colors)
    rgb_colors = np.array([mcolors.hsv_to_rgb([hue, 1, 0.9]) for hue in hues])
    rgb_colors = rgb_colors.reshape(n, n, 3)
    rgb_colors = np.roll(rgb_colors, shift=5, axis=1)

    # Draw
    draw_grid(rgb_colors)

    # Animate
    animate_grid(z)

if n_channels == 4:
    n_colors = n ** 2
    hues = np.linspace(0.5, 0.85, n_colors)
    colors = np.array([mcolors.hsv_to_rgb([hue, 1, 0.9]) for hue in hues])
    Qs = np.random.rand(colors.shape[0], 1)
    colors = np.hstack([colors, Qs])
    colors = colors.reshape(n, n, 4)
    colors = np.roll(colors, shift=5, axis=1)

    draw_grid(colors)
    animate_grid(z, tetra_image_mode='LMS_Q')